# 1.字节对编码（BPE）的主要思想

## 1.1 Bits和bytes

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(len(text))
print(byte_ary)

17
bytearray(b'This is some text')


In [2]:
ids = list(byte_ary)
print(len(ids))
print(ids)

17
[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


## 1.2 创建词汇表

## 1.3 BPE算法大览

# 2. BPE的实现

In [4]:
from collections import Counter, deque
from functools import lru_cache
import json
import re

class BPETokenizerSimple:
    def __init__(self):
        # 映射 token_id 到 token_str（例如：{11246: "some"}）
        self.vocab = {}
        # 映射 token_str 到 token_id（例如：{"some": 11246}）
        self.inverse_vocab = {}
        # BPE 合并字典：{(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}
        # GPT-2 用于合并的rank字典：{(string_A, string_B): rank}，其中较低的rank = 较高的优先级
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        从头开始训练 BPE 分词器。

        参数：
            text (str): 训练文本。
            vocab_size (int): 目标词汇表大小。
            allowed_special (set): 要包含的特殊令牌集。
        """

        # 预处理：将空格替换为 'Ġ'
        # 注意，Ġ 是 GPT-2 BPE 实现的一个特性
        # 例如，"Hello world" 可能被标记为 ["Hello", "Ġworld"]
        # （GPT-4 BPE 会将其标记为 ["Hello", " world"]）
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # 初始化词汇表
        unique_chars = [chr(i) for i in range(256)]

        # 扩展 unique_chars，包含处理后的文本中未包含的字符
        unique_chars.extend([char for char in sorted(set(processed_text)) if char not in unique_chars])
        
        # 可选：确保 'Ġ' 包含在内
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # 创建词汇表和逆词汇表
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # 添加允许的特殊token
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # 将文本转化为token ID
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE 步骤 1-3：反复查找并替换“出现次数最多”的字节对
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id
        
        # 使用合并后的token更新词汇表
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id
    
    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        从 OpenAI 的 GPT-2 文件加载预训练的词汇表和 BPE 合并。

        参数：
            vocab_path (str): 词汇文件路径（GPT-2 称其为 'encoder.json'）。
            bpe_merges_path (str): BPE 合并文件路径（GPT-2 称其为 'vocab.bpe'）。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # loaded_vocab 将 token_str 映射到 token_id
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # 如果词汇表中没有'\n'，则使用一个已有的token ID作为'\n'的占位符
        # 选择的这几个token ID都是在encode时不会被处理为子词的token
        if "\n" not in self.inverse_vocab:
            # 如果词汇表中存在"<|endoftext|>"，则使用它作为'\n'的占位符
            # 否则，使用"Ġ"或""作为'\n'的占位符
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # 加载BPE合并，并存储它们，并分配一个"rank"
        self.bpe_ranks = {}  # reset ranks
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # 如果token1或token2不在词汇表中，则跳过
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} as one token is not in the vocabulary.")


        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # loaded_vocab 将 token_str 映射到 token_id
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}
        # 加载BPE合并
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            # 如果有头部注释行，跳过它
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            for rank, line in enumerate(lines):
                pair = tuple(line.strip().split())
                if len(pair) != 2:
                    print(f"第 {rank+1} 行有超过 2 个条目：{line.strip()}")
                    continue
                token1, token2 = pair
                if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                    token_id1 = self.inverse_vocab[token1]
                    token_id2 = self.inverse_vocab[token2]
                    merged_token = token1 + token2
                    if merged_token in self.inverse_vocab:
                        merged_token_id = self.inverse_vocab[merged_token]
                        self.bpe_merges[(token_id1, token_id2)] = merged_token_id
                    else:
                        print(f"合并的token '{merged_token}' 未在词汇表中找到，跳过。")
                else:
                    print(f"跳过配对 {pair}，因为其中一个token不在词汇表中。")

    def encode(self, text, allowed_special=None):
        """
        将输入文本编码为token ID 列表。

        参数：
            text (str): 要编码的文本。
            allowed_special (set or None): 要包含的特殊token集。如果为None，则禁用特殊处理。

        返回：
            List[int]: token ID 列表。
        """
        token_ids = []

        # 如果允许特殊token处理，则构建一个正则表达式来匹配允许的特殊token
        if allowed_special is not None and len(allowed_special) > 0:
            # 构建一个正则表达式来匹配允许的特殊token
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )
    
            # 将文本根据特殊token进行分割，分别进行编码
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # 按正常处理编码前缀
    
                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
    
            text = text[last_index:]  # 剩余部分按正常处理
    
            # 检查剩余部分是否包含不允许的特殊token
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")

        # 如果没有任何特殊token，或者剩余文本在特殊token分割后：
        tokens = []
        # 将文本"\n"拆分为token，并处理空格
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:
                    tokens.append("Ġ" + word)
                elif j == 0:
                    tokens.append(word)
                else:
                    tokens.append("Ġ" + word)
    
        for token in tokens:
            if token in self.inverse_vocab:
                # token 已经存在于词汇表中
                token_ids.append(self.inverse_vocab[token])
            else:
                # 尝试通过 BPE 处理未登录词
                token_ids.extend(self.tokenize_with_bpe(token))
    
        return token_ids

    def tokenize_with_bpe(self, token):
        """
        使用 BPE 合并对单个token进行分词。

        参数：
            token (str): 要分词的token。

        返回：
            List[int]: 应用 BPE 后的token ID 列表。
        """
        # 将token分词为单个字符（作为初始token ID）
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # 若没有ranks信息，则使用下面的处理方法
        # 从左到右，依次合并token_ids中相邻的token，直到无法继续合并
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # Uncomment for educational purposes:
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # 跳过下一个token，因为它已经被合并
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # 若有ranks信息，则采用GPT-2-style的合并方法
        # 1) 将token_ids转换为字符串（为了适配bpe_ranks通过字符串存储key的形式）
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # 重复合并所有出现rank最低（优先级最高）的pair
        while True:
            # 收集所有相邻的pair
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # 找到出现次数最多的pair
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # 如果没有任何有效的合并pair，则结束
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # 合并所有出现的该pair
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # 位置i的pair，如果匹配(frist, second)，则进行合并
                if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                    new_symbols.append(first + second)  # 合并
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # 最后，将合并后的symbols转换成token IDs
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        将token ID 列表解码回字符串。

        参数：
            token_ids (List[int]): 要解码的token ID 列表。

        返回：
            str: 解码后的字符串。
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # 如果行尾没有空格，则添加一个空格
                decoded_string += token
            elif token.startswith("Ġ"):
                # 用空格替换 'Ġ'
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        将词汇表和 BPE 合并保存到 JSON 文件。

        参数：
            vocab_path (str): 保存词汇表的路径。
            bpe_merges_path (str): 保存 BPE 合并的路径。
        """
        # 保存词汇表
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump({k: v for k, v in self.vocab.items()}, file, ensure_ascii=False, indent=2)

        # 保存 BPE 合并作为字典列表
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        从 JSON 文件加载词汇表和 BPE 合并。

        参数：
            vocab_path (str): 词汇表文件路径。
            bpe_merges_path (str): BPE 合并文件路径。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # 加载 BPE 合并
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge['pair'])
                new_id = merge['new_id']
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("无效模式。选择 'most' 或 'least'。")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                dq.popleft()
            else:
                replaced.append(current)

        return replaced
        

### 3.1 训练、编码与解码

In [7]:
import os
import urllib.request

def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        with urllib.request.urlopen(url) as response, open(target_path, "wb") as out_file:
            out_file.write(response.read())
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")
    return target_path

verdict_path = download_file_if_absent(
    url=(
         "https://raw.githubusercontent.com/rasbt/"
         "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
         "the-verdict.txt"
    ),
    filename="the-verdict.txt",
    search_dirs="."
)

with open(verdict_path, "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

the-verdict.txt already exists in ./the-verdict.txt


In [8]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

In [9]:
print(len(tokenizer.vocab))

1000


In [10]:
print(len(tokenizer.bpe_merges))

742


In [11]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [12]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


In [13]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [14]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


In [15]:
for token_id in token_ids:
    print(f"{token_id} -> '{tokenizer.decode([token_id])}'")

424 -> 'Jack'
256 -> ' '
654 -> 'em'
531 -> 'br'
302 -> 'ac'
311 -> 'ed'
256 -> ' '
296 -> 'be'
97 -> 'a'
465 -> 'ut'
121 -> 'y'
595 -> ' through'
841 -> ' ar'
116 -> 't'
287 -> ' a'
466 -> 'nd'
256 -> ' '
326 -> 'li'
972 -> 'fe'
46 -> '.'


In [16]:
tokenizer.decode(tokenizer.encode("This is some text."))

'This is some text.'

In [17]:
# Save trained tokenizer
tokenizer.save_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [18]:
# Load tokenizer
tokenizer2 = BPETokenizerSimple()
tokenizer2.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [19]:
print(tokenizer2.decode(token_ids))

Jack embraced beauty through art and life.


&nbsp;
### 3.3 加载来自 OpenAI 的原始 GPT-2 BPE 分词器

In [20]:
import os
import urllib.request

def download_file_if_absent(url, filename):
    if not os.path.exists(filename):
        try:
            with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
                out_file.write(response.read())
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed to download {filename}. Error: {e}")
    else:
        print(f"{filename} already exists")

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

for url, filename in files_to_download.items():
    download_file_if_absent(url, filename)

vocab.bpe already exists
encoder.json already exists


In [21]:
tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path="encoder.json", bpe_merges_path="vocab.bpe"
)

In [22]:
len(tokenizer_gpt2.vocab)

50257

In [23]:
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print(token_ids)

[1212, 318, 617, 2420]


In [24]:
print(tokenizer_gpt2.decode(token_ids))

This is some text


In [25]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")
tik_tokenizer.encode(input_text)

/Users/young/project/llmProject/LLMs-from-scratch-CN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[1212, 318, 617, 2420]